In [3]:
import pandas as pd
from mendeleev.fetch import fetch_table
import re

In [4]:
data_df = pd.read_csv("20231204v1.csv")
elem_df = fetch_table("elements")
elem_df['symbol']

0       H
1      He
2      Li
3      Be
4       B
       ..
113    Fl
114    Mc
115    Lv
116    Ts
117    Og
Name: symbol, Length: 118, dtype: object

In [5]:
def parse_sample(data):
    parsed_data = []
    elem_df = fetch_table("elements")
    all_elems = elem_df['symbol']
    pat = re.compile("|".join(all_elems.tolist()))
    for s, sample in data.iterrows():
        comp = sample["True Composition"]
        match = re.findall(pat, comp)
        stoich = re.split(pat, comp)[1:]
        dix = {}
        dix["Space Group"] = sample["Space group number"]
        dix["a"] = sample["a"]
        dix["b"] = sample["b"]
        dix["c"] = sample["c"]
        dix["alpha"] = sample["alpha"]
        dix["beta"] = sample["beta"]
        dix["gamma"] = sample["gamma"]
        for e in all_elems:
            dix[e] = 0
        for e, f in zip(match, stoich):
            match = re.match(r"([a-z]+)([0-9]+)", f, re.I)
            if match:
                items = match.groups()
                dix[e + items[0]] = float(items[1])
            else:
                dix[e] = float(f)
        dix["IC"] = sample["Ionic conductivity (S cm-1)"]
        parsed_data.append(dix)

    return pd.DataFrame(parsed_data)
    

In [6]:
final = parse_sample(data_df)
final
final.to_csv("20231204v2.csv", index=False)